In [1]:
import deepchem as dc
import numpy as np
import pandas as pd

2023-01-07 21:37:39.994290: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-07 21:37:40.855261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/lac/anaconda3/envs/deepchem_env/lib/
2023-01-07 21:37:40.855390: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/lac/anaconda3/envs/deepchem_env/lib/
2023-01-07 21:37:40.855398: W tensorflow/c

In [2]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='ECFP')
train_dataset, valid_dataset, test_dataset = datasets

In [3]:
model = dc.models.RobustMultitaskClassifier(n_tasks=12, 
                                            n_features=1024, 
                                            layer_sizes=[1024])

2023-01-07 21:37:45.398219: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 21:37:45.405631: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 21:37:45.405938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-07 21:37:45.406656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [4]:
model.fit(train_dataset, nb_epoch=10)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
print('Training set score:', model.evaluate(train_dataset, [metric], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric], transformers))

2023-01-07 21:37:57.292305: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x55e86214c170 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-01-07 21:37:57.292329: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce GTX 1060, Compute Capability 6.1
2023-01-07 21:37:57.298031: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-01-07 21:37:57.443304: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Training set score: {'mean-roc_auc_score': 0.9661840010717327}
Test set score: {'mean-roc_auc_score': 0.6735673068282931}


In [5]:
tasks, datasets, transformers = dc.molnet.load_tox21(featurizer='GraphConv')
train_dataset, valid_dataset, test_dataset = datasets

In [6]:
print(train_dataset)
print(valid_dataset)
print(test_dataset)
print(tasks)
print(transformers)

<DiskDataset X.shape: (6264,), y.shape: (6264, 12), w.shape: (6264, 12), task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>
<DiskDataset X.shape: (783,), y.shape: (783, 12), w.shape: (783, 12), ids: ['N#C[C@@H]1CC(F)(F)CN1C(=O)CNC1CC2CCC(C1)N2c1ncccn1'
 'CN(C)C(=O)NC1(c2ccccc2)CCN(CCC[C@@]2(c3ccc(Cl)c(Cl)c3)CCCN(C(=O)c3ccccc3)C2)CC1'
 'CSc1nnc(C(C)(C)C)c(=O)n1N' ...
 'O=C(O[C@H]1CN2CCC1CC2)N1CCc2ccccc2[C@@H]1c1ccccc1'
 'C#C[C@]1(O)CC[C@H]2[C@@H]3CCC4=CC(=O)CC[C@@H]4[C@H]3C(=C)C[C@@]21CC'
 'NC(=O)C(c1ccccc1)(c1ccccc1)[C@@H]1CCN(CCc2ccc3c(c2)CCO3)C1'], task_names: ['NR-AR' 'NR-AR-LBD' 'NR-AhR' ... 'SR-HSE' 'SR-MMP' 'SR-p53']>
<DiskDataset X.shape: (784,), y.shape: (784, 12), w.shape: (784, 12), ids: ['CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.CC1(C)S[C@@H]2[C@H](NC(=O)Cc3ccccc3)C(=O)N2[C@H]1C(=O)O.c1ccc(CNCCNCc2ccccc2)cc1'
 'CC(C)(c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1)c1ccc(Oc2ccc3c(c2)C(=O)OC3=O)cc1'
 'Cc1cc(C(C)(C)C)c(O)c(C)c1Cn1c(=O)n(Cc2c(C)cc(C(C)(C)

In [7]:
n_tasks = len(tasks)
batch_size = 128

In [8]:
model = dc.models.GraphConvModel(n_tasks, 
                                 mode='classification', 
                                 batch_size=batch_size,
                                 model_dir='../models')
model.fit(train_dataset, nb_epoch=50)

0.2595729446411133

In [9]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score, np.mean)
print('Training set score:', model.evaluate(train_dataset, [metric], transformers))
print('Test set score:', model.evaluate(test_dataset, [metric], transformers))

Training set score: {'mean-roc_auc_score': 0.9731465873064765}
Test set score: {'mean-roc_auc_score': 0.7095828758533238}


In [10]:
pred = model.predict(test_dataset)

In [11]:
# [toxic, nontoxic] for each class
pred

array([[[8.9169800e-01, 1.0830200e-01],
        [9.9424642e-01, 5.7536471e-03],
        [8.4670073e-01, 1.5329923e-01],
        ...,
        [5.6693423e-01, 4.3306577e-01],
        [9.6280432e-01, 3.7195709e-02],
        [9.6311665e-01, 3.6883343e-02]],

       [[9.5142287e-01, 4.8577081e-02],
        [9.9920100e-01, 7.9901505e-04],
        [9.2275774e-01, 7.7242233e-02],
        ...,
        [4.7724807e-01, 5.2275199e-01],
        [5.1812518e-01, 4.8187479e-01],
        [1.5451497e-01, 8.4548503e-01]],

       [[9.9851912e-01, 1.4808973e-03],
        [9.9993205e-01, 6.7964538e-05],
        [9.9085337e-01, 9.1466326e-03],
        ...,
        [8.1599164e-01, 1.8400839e-01],
        [5.6524307e-01, 4.3475699e-01],
        [9.8482311e-01, 1.5176857e-02]],

       ...,

       [[9.5263785e-01, 4.7362126e-02],
        [7.6178819e-01, 2.3821183e-01],
        [3.4113908e-01, 6.5886092e-01],
        ...,
        [9.6663976e-01, 3.3360250e-02],
        [8.4752208e-01, 1.5247795e-01],
        [

In [38]:
pred.shape

(784, 12, 2)

In [48]:
y_true = test_dataset.y
#print(y_true.shape)
print(y_true[0:10])
print(pred[3])


[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[0.88619334 0.11380665]
 [0.9880189  0.01198121]
 [0.8082292  0.19177072]
 [0.97490966 0.02509035]
 [0.17723855 0.8227615 ]
 [0.92856276 0.07143722]
 [0.9923004  0.00769958]
 [0.49192783 0.5080722 ]
 [0.8349025  0.16509743]
 [0.9891577  0.01084228]
 [0.97145295 0.02854702]
 [0.97891515 0.0210848 ]]


In [54]:
metric = dc.metrics.roc_auc_score
cummulative_score = 0.
for i in range(n_tasks):
    score = metric(dc.metrics.to_one_hot(y_true[:,i]), pred[:,i])
    print(tasks[i], score)
    cummulative_score += score

NR-AR 0.7134889182445325
NR-AR-LBD 0.721499828001376
NR-AhR 0.7846035436039207
NR-Aromatase 0.6635584168134183
NR-ER 0.599779911964786
NR-ER-LBD 0.7587842476440118
NR-PPAR-gamma 0.7434979718444286
SR-ARE 0.5909171883748154
SR-ATAD5 0.7698018803211879
SR-HSE 0.7270417737232597
SR-MMP 0.7494246608527132
SR-p53 0.6925961688514357


In [55]:
avg_score = cummulative_score/n_tasks
avg_score

0.7095828758533238

#### GraphConvModel builing from scratch

In [33]:
from deepchem.models.layers import GraphConv, GraphPool, GraphGather
import tensorflow as tf
import tensorflow.keras.layers as layers

batch_size = 100

class CustGraphConvModel(tf.keras.Model):

  def __init__(self):
    super(CustGraphConvModel, self).__init__()
    self.gc1 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm1 = layers.BatchNormalization()
    self.gp1 = GraphPool()

    self.gc2 = GraphConv(128, activation_fn=tf.nn.tanh)
    self.batch_norm2 = layers.BatchNormalization()
    self.gp2 = GraphPool()

    self.dense1 = layers.Dense(256, activation=tf.nn.tanh)
    self.batch_norm3 = layers.BatchNormalization()
    self.readout = GraphGather(batch_size=batch_size, activation_fn=tf.nn.tanh)

    self.dense2 = layers.Dense(n_tasks*2)
    self.logits = layers.Reshape((n_tasks, 2))
    self.softmax = layers.Softmax()

  def call(self, inputs):
    gc1_output = self.gc1(inputs)
    batch_norm1_output = self.batch_norm1(gc1_output)
    gp1_output = self.gp1([batch_norm1_output] + inputs[1:])

    gc2_output = self.gc2([gp1_output] + inputs[1:])
    batch_norm2_output = self.batch_norm1(gc2_output)
    gp2_output = self.gp2([batch_norm2_output] + inputs[1:])

    dense1_output = self.dense1(gp2_output)
    batch_norm3_output = self.batch_norm3(dense1_output)
    readout_output = self.readout([batch_norm3_output] + inputs[1:])

    logits_output = self.logits(self.dense2(readout_output))
    return self.softmax(logits_output)



In [34]:
cust_model = dc.models.KerasModel(CustGraphConvModel(), loss=dc.models.losses.CategoricalCrossEntropy())

In [35]:
test_dataset.X[0]

In [36]:
from deepchem.metrics import to_one_hot
from deepchem.feat.mol_graphs import ConvMol

def data_generator(dataset, epochs=1):
  for ind, (X_b, y_b, w_b, ids_b) in enumerate(dataset.iterbatches(batch_size, epochs,
                                                                   deterministic=False, pad_batches=True)):
    multiConvMol = ConvMol.agglomerate_mols(X_b)
    inputs = [multiConvMol.get_atom_features(), multiConvMol.deg_slice, np.array(multiConvMol.membership)]
    for i in range(1, len(multiConvMol.get_deg_adjacency_lists())):
      inputs.append(multiConvMol.get_deg_adjacency_lists()[i])
    labels = [to_one_hot(y_b.flatten(), 2).reshape(-1, n_tasks, 2)]
    weights = [w_b]
    yield (inputs, labels, weights)

In [37]:
cust_model.fit_generator(data_generator(train_dataset, epochs=50))

0.22766029357910156

In [27]:
print('Training set score:', cust_model.evaluate_generator(data_generator(train_dataset), [metric], transformers))
print('Test set score:', cust_model.evaluate_generator(data_generator(test_dataset), [metric], transformers))

Training set score: {'mean-roc_auc_score': 0.4656707930765321}
Test set score: {'mean-roc_auc_score': 0.5067519229695513}
